# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vac_df=pd.read_csv('weather_data.csv').drop('City_ID', axis=1)
vac_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.80,-68.30,32.00,95,90,23.04,AR,1595810577
1,Saskylakh,71.92,114.08,38.73,94,100,18.34,RU,1595810577
2,Jamestown,42.10,-79.24,77.00,65,1,5.82,US,1595810404
3,Lüderitz,-26.65,15.16,60.80,63,0,12.75,NaN,1595810531
4,Mpulungu,-8.76,31.11,65.32,67,56,7.83,ZM,1595810578


In [4]:
len(vac_df)

574

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
locations = vac_df[['Lat', 'Lng']]
humidity=vac_df['Humidity']
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
vac_drill_df=vac_df.loc[(vac_df['Max Temp']>70) & (vac_df['Max Temp']<80)]
len(vac_drill_df)

149

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df=vac_drill_df.copy()
hotel_df['Hotel Name'] = ""
len(hotel_df)
hotel_df=hotel_df.reset_index()
hotel_df=hotel_df.drop('index', axis=1)

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_radius = 5000
target_type = "hotel"
# set up a parameters dictionary
params = {
    "radius": target_radius,
    "keyword": target_type,
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'

    response=requests.get(base_url, params=params)
    places_data=response.json()
    try:
        hotel_df.loc[index, 'Hotel Name']=places_data['results'][0]['name']
        print(f"hotel name for {row['City']} is {places_data['results'][0]['name']}")
        
    except:
        print(f"Failed hotel lookup for {row['City']}")
  

hotel name for Jamestown is Chautauqua Harbor Hotel
Failed hotel lookup for Kayiry
hotel name for Bijie is Four Points by Sheraton Bijie
hotel name for Hobyo is Osmani Hotel
hotel name for Vaini is Paradise First Hotel
hotel name for Rikitea is Pension Bianca & Benoit
hotel name for Bengkulu is Hotel Santika Bengkulu
hotel name for São Filipe is Colonial Guest House of Fogo
hotel name for Alappuzha is Dona House Boats at Alappuzha (Alleppey Backwaters)
hotel name for Lom Sak is WORACHAT BOUTIQUE HOTEL
hotel name for Guerrero Negro is Hotel Los Caracoles
hotel name for Coahuayana Viejo is Hotel Los Arcos
hotel name for Muroto-misakicho is Ake no Hoshi
hotel name for Yangi Marg`ilon is Hotel Mehmon Saroy
hotel name for Sinnamary is L'oiseau de Paradis
Failed hotel lookup for Kogon
hotel name for Lethem is Takutu Hotel
hotel name for Matara is Villa Océane - Madiha Mirissa
hotel name for Shahreza is خانه مسافر اسپاد
hotel name for Acaraú is Castelo Encantado
hotel name for Avarua is Pacif

In [13]:
hotel_df.head()
hotel_df=hotel_df.loc[hotel_df['Hotel Name']!='']


In [14]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Jamestown,42.10,-79.24,77.00,65,1,5.82,US,1595810404,Chautauqua Harbor Hotel
2,Bijie,27.31,105.29,72.03,66,0,4.00,CN,1595810579,Four Points by Sheraton Bijie
3,Hobyo,5.35,48.53,77.38,75,3,18.39,SO,1595810580,Osmani Hotel
4,Vaini,-21.20,-175.20,75.20,78,75,10.29,TO,1595810524,Paradise First Hotel
5,Rikitea,-23.12,-134.97,70.32,79,0,19.89,PF,1595810583,Pension Bianca & Benoit


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig=gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))